In [1]:
import numpy as np
import pandas as pd
import re
from collections import Counter
import pickle
import random

### Get phrase counts

In [3]:
with open('../pickles/counter_Z.pickle', 'rb') as inputfile:
    a = pickle.load(inputfile)
with open('../pickles/counter_Z2.pickle', 'rb') as inputfile:
    b = pickle.load(inputfile)
with open('../pickles/counter_Z3.pickle', 'rb') as inputfile:
    c = pickle.load(inputfile)
with open('../pickles/counter_Z4.pickle', 'rb') as inputfile:
    d = pickle.load(inputfile)
with open('../pickles/counter_Z5.pickle', 'rb') as inputfile:
    e = pickle.load(inputfile)
with open('../pickles/counter_Z6.pickle', 'rb') as inputfile:
    f = pickle.load(inputfile)

In [4]:
phrase_counter = a + b + c + d + e + f

In [5]:
top300k = phrase_counter.most_common(300000)

### Get word counts

In [6]:
import scipy.sparse

In [7]:
count_vect = scipy.sparse.load_npz('../data/count_vect.npz')
feat_vect = np.load('../data/vect_feat.npy')

In [8]:
count_dict = {'word': feat_vect, 'count':count_vect.sum(axis = 0).tolist()[0]}

In [9]:
count_df = pd.DataFrame(count_dict)

In [10]:
count_df = count_df.set_index('word', drop=True)

In [11]:
count_df.head()

,count
word,
00,2463
000,60512
0000,26
00000,4
0000000,1


In [12]:
freq_dict = count_df.to_dict()['count']

### Make df

In [13]:
arr = np.array(top300k)
df300k = pd.DataFrame(data=arr, columns=['phrases', 'counts'])

In [14]:
split_df = df300k.phrases.str.split(' ', expand=True)
split_df['phrases'] = df300k.phrases
split_df['phrase_count'] = df300k.counts.astype(int)
split_df.rename(columns={0:'w_one', 1:'w_two'}, inplace=True)

In [15]:
split_df.shape

(300000, 4)

In [16]:
split_df.head(5)

,w_one,w_two,phrases,phrase_count
0,last,year,last year,19906
1,last,week,last week,15305
2,years,ago,years ago,14025
3,first,time,first time,10717
4,social,media,social media,10634


In [17]:
def isEnglish(s):
    return s.isascii()

In [18]:
not_english = split_df[~split_df.phrases.apply(lambda x: isEnglish(x))]

In [19]:
split_df = split_df[split_df.phrases.apply(lambda x: isEnglish(x))]

In [20]:
split_df = split_df[split_df.w_one.apply(lambda x: len(x)) > 2]
split_df = split_df[split_df.w_two.apply(lambda x: len(x)) > 2]

In [21]:
split_df['freq_one'] = split_df.w_one.apply(lambda x: freq_dict[x])
split_df['freq_two'] = split_df.w_two.apply(lambda x: freq_dict[x])

In [22]:
split_df['mean_freq_ratio'] = split_df.phrase_count / ((split_df.freq_one + split_df.freq_two)/2)

In [23]:
infinity = split_df[split_df.mean_freq_ratio > 10].index
split_df.drop(infinity, axis=0, inplace=True)

In [27]:
top10k = split_df.mean_freq_ratio.nlargest(10000).index

In [28]:
top10k

Int64Index([288873,  65357,  72692,  72641, 133769, 257426,  13521, 106351,
            282562,  43604,
            ...
            274039, 288951,  22975,  67386,   6786, 116206, 152514,   4519,
             38900,  86279],
           dtype='int64', length=10000)

In [30]:
new_df = split_df.loc[top10k,:]

In [32]:
new_df.head()

,w_one,w_two,phrases,phrase_count,freq_one,freq_two,mean_freq_ratio
288873,hemolytic,uremic,hemolytic uremic,5,5,5,1.000000
65357,hoi,polloi,hoi polloi,21,24,23,0.893617
72692,vas,deferens,vas deferens,20,28,20,0.833333
72641,situs,inversus,situs inversus,20,25,24,0.816327
133769,myocardial,infarction,myocardial infarction,11,13,14,0.814815


In [35]:
new_df[(new_df.mean_freq_ratio < .3) & (new_df.mean_freq_ratio > .2)]

,w_one,w_two,phrases,phrase_count,freq_one,freq_two,mean_freq_ratio
2565,pepper,spray,pepper spray,301,1029,982,0.299354
15222,softwood,lumber,softwood lumber,75,99,404,0.298211
2555,fetal,tissue,fetal tissue,303,823,1247,0.292754
105,receive,updates,receive updates,2113,9569,4896,0.292153
5055,densely,populated,densely populated,180,318,918,0.291262
176588,bete,noire,bete noire,9,11,51,0.290323
1843,secured,unconditional,secured unconditional,384,2037,615,0.289593
21056,haute,couture,haute couture,57,162,232,0.289340
51308,prefrontal,cortex,prefrontal cortex,27,31,157,0.287234
97263,fidget,spinners,fidget spinners,15,44,61,0.285714


# Stop 

### Get Part of Speech

In [10]:
# lem = WordNetLemmatizer()
# lem_words = {}
# for x in df.word.values:    
#     pos = nltk.pos_tag([x])
#     for w,p in pos:
#         p_new = get_wordnet_pos(p)
#         d = lem.lemmatize(w,p_new)
#     lem_words[x] = d    

In [11]:
# def get_wordnet_pos(treebank_tag):

#     if treebank_tag.startswith('J'):
#         return wordnet.ADJ
#     elif treebank_tag.startswith('V'):
#         return wordnet.VERB
#     elif treebank_tag.startswith('N'):
#         return wordnet.NOUN
#     elif treebank_tag.startswith('R'):
#         return wordnet.ADV
#     else:
#         return wordnet.NOUN

In [12]:
# for w,p in pos:
#     p_new = get_wordnet_pos(p)
#     lemmed_words.append(lem.lemmatize(w,p_new))
#     lemmed_sentence += (w + ' ')

In [13]:
# w1 = split_df.w_one.replace(lem_words)

In [14]:
# w2 = split_df.w_two.replace(lem_words)

In [15]:
# split_df['w1'] = w1
# split_df['w2'] = w2

### We have DF with row for every phrase. 
### Now make one with every word occuring in all phrases
### Count how often each word occurs as first, how often it occurs second

In [47]:
w1_count = Counter(split_df.w_one.values)
w2_count = Counter(split_df.w_two.values)

In [48]:
word_count = w1_count + w2_count
words_inc = list(set(word_count.keys()))
df = pd.DataFrame(words_inc, columns=['word'])
df['start_count'] = df.word.apply(lambda x: w1_count[x])
df['end_count'] = df.word.apply(lambda x: w2_count[x])

In [49]:
df.shape

(22842, 3)

In [ ]:
df

In [20]:
nix_words = ['made', 'make', 'making', 'liking', 'looking', 'like', 'liked', 'likes', 
             'many', 'even', 'though', 'look', 'looks', 'looked', 'take', 'took', 'know', 
             'knew', 'knows', 'say', 'said', 'go', 'got', 'much', 'often', 'who', 'whom', 
             'whose', 'their', 'theirs', 'dozen', 'might', 'may', 'never', 'also', 'still',
             'expressed', 'went', 'expresses', 'express', 'expressing', 'saying', 'u']

'''
when, where, why, that

'''
pass

In [21]:
df = df[~df.word.isin(nix_words)]
split_df = split_df[~split_df.w_two.isin(nix_words)]
split_df = split_df[~split_df.w_one.isin(nix_words)]

In [22]:
split_df.shape

(89479, 3)

In [24]:
df['start_count'] = df.word.apply(lambda x: split_df[split_df.w_one==x]['w_one'].count())

In [25]:
df['end_count'] = df.word.apply(lambda x: split_df[split_df.w_two==x]['w_two'].count())

In [26]:
df = df[(df.start_count > 3)&(df.end_count > 3)]
# print(df.shape)
# df = df[df.end_count > 7]
# print(df.shape)
# df = df[df.]

In [28]:
split_df = split_df[split_df.w_one.isin(df.word.values)]
split_df = split_df[split_df.w_two.isin(df.word.values)]

In [29]:
split_df.shape

(53133, 3)

In [31]:
# csv = split_df.loc[:,['phrases', 'w1', 'w2']].sort_values(['w1'])

In [32]:
# csv.to_csv('phrase_list.csv')

maybe remove:
made, make, like, many, even, though, first, look, time, (prepositions??) , take, used, using, use, say, know, le, get, much, go, got, often, who, whose, dozen, might, may, never, also, say, seem, still, express, 

In [33]:
df.shape

(2197, 3)

In [34]:
# df['before'] = df.word.apply(lambda x: split_df[split_df.w_two == x].w_one.values)

In [36]:
df['after'] = df.word.apply(lambda x: split_df[split_df.w_one == x].w_two.values)

In [37]:
df.end_count.sum()

69003

### Test lem word replacement

In [38]:
# df['word_test'] = df.word

In [39]:
# df.word_test = df.word_test.replace(lem_words)
# df.word_test.shape

In [40]:
# ct_dict = df.groupby('word_test').count().word.to_dict()

In [41]:
# df['repeats'] = df.word_test.replace(ct_dict)

In [42]:
df.head()

,word,start_count,end_count,after
6,management,24,26,"[team, company, style, firm, system, fee, skil..."
8,normal,43,12,"[life, people, time, person, level, human, rul..."
13,written,23,27,"[statement, testimony, response, question, boo..."
14,wounded,9,11,"[people, soldier, officer, veteran, man, hundr..."
21,born,10,17,"[citizen, outside, without, classified, female..."


In [43]:
df['all_words'] = df.start_count + df.end_count

In [44]:
df.sort_values(by=['all_words'], ascending=False)

,word,start_count,end_count,after,all_words
7469,new,901,225,"[book, administration, president, study, repor...",1126
1874,people,528,504,"[want, think, around, feel, died, dead, living...",1032
8845,political,590,201,"[party, analyst, system, leader, class, news, ...",791
372,first,638,70,"[time, place, reported, step, half, round, thi...",708
4815,another,441,153,"[way, person, example, man, country, woman, re...",594
120,last,174,405,"[night, time, summer, season, fall, weekend, d...",579
1129,without,276,300,"[fear, ever, evidence, knowing, charge, gettin...",576
8929,state,285,234,"[law, government, medium, official, attorney, ...",519
842,around,46,469,"[town, since, campus, half, people, long, time...",515
12110,since,80,410,"[last, taking, early, become, leaving, late, b...",490


In [45]:
### Examples of why this lemma doesn't work -- adopted home --> adopt home ; disappointing result --> disappoint result

In [46]:
### don't think this adds anything new, same as other column...

#df['num_after'] = df.after.str.len()

### Consider drops

In [47]:
new_df = df.set_index(df.word).drop('word', axis=1)
new_df.head(20)

,start_count,end_count,after,all_words
word,,,,
management,24,26,"[team, company, style, firm, system, fee, skil...",50
normal,43,12,"[life, people, time, person, level, human, rul...",55
written,23,27,"[statement, testimony, response, question, boo...",50
wounded,9,11,"[people, soldier, officer, veteran, man, hundr...",20
born,10,17,"[citizen, outside, without, classified, female...",27
star,24,58,"[power, player, turn, system, witness, turned,...",82
provides,18,8,"[free, service, expert, additional, support, a...",26
global,162,25,"[economy, market, financial, temperature, trad...",187
support,89,185,"[among, group, behind, system, staff, network,...",274


In [61]:
new_df.head(50).to_csv('top50.csv')

In [48]:
# w2_filt = { x: count for x, count in w2_count.items() if w2_count[x] >= 10 }
# w1_filt = { x: count for x, count in w1_count.items() if w1_count[x] >= 10 }
# print(len(w2_filt))
# print(len(w1_filt))

In [49]:
temp_df = pd.DataFrame([df.word, df.after]).T

In [50]:
temp_df = temp_df.set_index(temp_df.word).drop('word', axis=1)

In [51]:
word_dict0 = temp_df.T.to_dict('list')

In [52]:
word_dict = {k:v[0].tolist() for k, v in word_dict0.items()}

In [63]:
with open('temp_dict.pickle', 'wb') as outputfile:
    pickle.dump(word_dict, outputfile)

In [53]:
### pick key randomly
word_list = []
word1 = random.choice(list(word_dict.keys()))
word_list.append(word1)

In [54]:
def pick_new(l_word, word_list):
    rng = list(range(len(word_dict[l_word])))
    random.shuffle(rng)
    random.shuffle(rng)
    choices = list(np.array(word_dict[l_word])[rng])
    for i in range(len(choices)):
        l = len(choices)
        if l > 0:
            if choices[i] in word_dict.keys():
                if choices[i] not in word_list:
                    new_word = choices[i]
                    word_list.append(new_word)
                    break
                else:
                    choices.remove(choices[i])
        else:
            raise InputError('No choices left')
    return new_word, word_list


In [ ]:
'''
Eliminate repeats to avoid :

Dogs 
Walk
Dogs
Walk
...

'''
pass

In [55]:
def pick_set():
    word_list = []
    word1 = random.choice(list(word_dict.keys()))
    word_list.append(word1)
    
    word2, word_list = pick_new(word1, word_list)
    word3, word_list = pick_new(word2, word_list)
    word4, word_list = pick_new(word3, word_list)
    word5, word_list = pick_new(word4, word_list)
    word6, word_list = pick_new(word5, word_list)
    word7, word_list = pick_new(word6, word_list)
    word8, word_list = pick_new(word7, word_list)
    word9, word_list = pick_new(word8, word_list)
    word10, word_list = pick_new(word9, word_list)
    return word_list


In [58]:
list1 = pick_set()
list2 = pick_set()
list3 = pick_set()
list4 = pick_set()
list5 = pick_set()
list6 = pick_set()
list7 = pick_set()
list8 = pick_set()
list9 = pick_set()
list10 = pick_set()
list11 = pick_set()
list12 = pick_set()
list13 = pick_set()

In [62]:
print(list1)
print(list2)
print(list3)
print(list4)
print(list5)
print(list6)
print(list7)
print(list8)
print(list9)
print(list10)
print(list11)
print(list12)
print(list13)

['negative', 'statement', 'later', 'allowed', 'people', 'fought', 'hard', 'thing', 'become', 'president']
['terrorist', 'crime', 'rose', 'gold', 'standard', 'regarding', 'immigration', 'enforcement', 'activity', 'outside']
['eating', 'dinner', 'party', 'already', 'talking', 'politics', 'right', 'man', 'started', 'training']
['simply', 'call', 'ended', 'without', 'trying', 'new', 'history', 'department', 'need', 'today']
['comment', 'came', 'away', 'money', 'talk', 'begin', 'work', 'closely', 'followed', 'suit']
['seem', 'increasingly', 'turned', 'head', 'next', 'meet', 'weekly', 'column', 'called', 'cop']
['positive', 'note', 'taken', 'inside', 'story', 'straight', 'man', 'fell', 'sharply', 'cut']
['bankruptcy', 'last', 'winter', 'blue', 'blood', 'test', 'run', 'without', 'anything', 'unusual']
['household', 'item', 'including', 'cutting', 'deal', 'since', 'news', 'agenda', 'without', 'receiving']
['shot', 'behind', 'bar', 'elected', 'lawmaker', 'last', 'good', 'effect', 'immediately',

In [34]:
word2, word_list = pick_new(word1, word_list)
word2

'released'

In [35]:
word3, word_list = pick_new(word2, word_list)
word3

'immediately'

In [36]:
word4, word_list = pick_new(word3, word_list)
word4

'upon'

In [37]:
word5, word_list = pick_new(word4, word_list)
word5

'release'

In [38]:
word6, word_list = pick_new(word5, word_list)
word6

'video'

In [39]:
word7, word_list = pick_new(word6, word_list)
word7

'game'

In [40]:
word8, word_list = pick_new(word7, word_list)
word8

'behind'

In [41]:
word9, word_list = pick_new(word8, word_list)
word9

'wall'

In [42]:
word10, word_list = pick_new(word9, word_list)
word10

'around'